# Maxwell eigenvalue problem
We solve the Maxwell eigenvalue problem

$$
\int \operatorname{curl} u \, \operatorname{curl} v
= \lambda \int u v 
$$

for $u, v \; \bot \; \nabla H^1$
using a PINVIT solver from the ngsolve solvers module.

The orthogonality to gradient fields is important to eliminate the huge number of zero Eigenvalues. The orthogonal sub-space is implemented using a Poisson projection:

$$
P u = u - \nabla \Delta^{-1} \operatorname{div} u
$$

The algorithm and example is take form the Phd thesis of Sabine Zaglmayr, p 145-150.

In [5]:
import netgen.gui
from netgen.csg import *
from ngsolve import *

geom = CSGeometry()

cube1 = OrthoBrick(Pnt(-1,-1,-1), Pnt(1,1,1))
cube2 = OrthoBrick(Pnt(0,0,0), Pnt(2,2,2)) 
fichera = cube1-cube2
geom.Add (fichera)

# all edges defined by intersection of faces of 
# cube2 and cube2 are marked for geometric edge refinement
geom.SingularEdge (cube2,cube2, 1)
geom.SingularPoint (cube2,cube2,cube2, 1)

mesh = Mesh(geom.GenerateMesh(maxh=0.5))
# two levels of hp-refinement
mesh.RefineHP(2)
Draw (mesh)

In [9]:
SetHeapSize(100*1000*1000)

fes = HCurl(mesh, order=3)
print ("ndof =", fes.ndof)
u,v = fes.TnT()

a = BilinearForm(fes)
a += curl(u)*curl(v)*dx

m = BilinearForm(fes)
m += u*v*dx

apre = BilinearForm(fes)
apre += curl(u)*curl(v)*dx + u*v*dx
pre = Preconditioner(apre, "direct", inverse="sparsecholesky")

ndof = 27931


In [10]:
with TaskManager():
    a.Assemble()
    m.Assemble()
    apre.Assemble()

    # build gradient matrix as sparse matrix (and corresponding scalar FESpace)
    gradmat, fesh1 = fes.CreateGradient()
    
    
    gradmattrans = gradmat.CreateTranspose() # transpose sparse matrix
    math1 = gradmattrans @ m.mat @ gradmat   # multiply matrices 
    math1[0,0] += 1     # fix the 1-dim kernel
    invh1 = math1.Inverse(inverse="sparsecholesky")

    # build the Poisson projector with operator Algebra:
    proj = IdentityMatrix() - gradmat @ invh1 @ gradmattrans @ m.mat

    projpre = proj @ pre.mat

    evals, evecs = solvers.PINVIT(a.mat, m.mat, pre=projpre, num=12, maxit=20)

0 : [4.757435661995881, 14.073620183520768, 18.356185824162093, 32.48563933256732, 41.43408846665329, 46.11881313652462, 50.35356422491064, 54.69694359276777, 66.75250536363436, 73.26690303619192, 79.28802019431366, 91.86519987389241]
1 : [3.227920132689829, 6.114594419385947, 6.302950873719183, 11.802066814986082, 12.460645617187396, 13.595914700608994, 14.578251848776796, 17.448744449338673, 18.53642644520641, 20.30435827271497, 21.4603175928845, 25.91058659789061]
2 : [3.220824395828742, 5.890102993628636, 5.911388622685046, 10.812954375751298, 10.996247058839044, 11.741931314556112, 12.710773432839327, 13.710917478918283, 13.933121005154423, 14.372340310398073, 15.714492618355951, 17.010116307672924]
3 : [3.2207323571634987, 5.881057419399865, 5.882680632007509, 10.70424051382207, 10.75681761177079, 11.053662957358673, 12.39603257248877, 12.827350854340972, 13.48873040946222, 13.625744380170442, 14.034980691415496, 15.048771500563236]
4 : [3.220731107910896, 5.880700839319016, 5.88

In [12]:
print ("Eigenvalues")
for lam in evals: 
    print (lam)

Eigenvalues
3.220731090311838
5.880684362321143
5.880702623195154
10.68703460393453
10.694664639568135
10.694878359383914
12.319937398910547
12.320469436651502
13.35917900081837
13.424525068835669
13.425208606308141
14.2183206756301


In [14]:
Draw (mesh)
gfu = GridFunction(fes, multidim=len(evecs))
for i in range(len(evecs)):
    gfu.vecs[i].data = evecs[i]
Draw (Norm(gfu), mesh, "u", sd=4)